### Adding noise to the EAGLE simulations
This goal of this notebook is to explore adding noise to the EAGLE simulation to make mock observations.  The main part of this notebook doesn't even use the simulations, it just explores the sources of noise as added to a pretend data set (a circle on a blank background).

We will go through the different types of noise so that it is clear which one dominates the signal.  In particular, we want to explore when read out noise trades out with the sky background as the dominant noise source.

Once we times the photon signals (object and background) by the QE, they are counted in electrons instead of photons.

### Sources of Poisson noise:
The shot noise, dark current noise, and sky background noise are all Poisson processes.  Therefore can draw from a Poisson or Gaussian (large number statistics) distribution with a sigma that is the square root of the mean value for each background source to draw random values for the noise.

> For small photon counts, photon noise is generally dominated by other signal-independent sources of noise, and
for larger counts, the central limit theorem ensures that the Poisson distribution approaches a Gaussian.

> The ratio of signal to photon noise grows with the square root
of the number of photons captured, √ λt. <br>
( https://people.csail.mit.edu/hasinoff/pubs/hasinoff-photon-2012-preprint.pdf )

Poission_noise = sigma = SQRT(meanvalue_Poisson)

By the Central Limit Theorem, the Poisson distribution can be approximated as a Gaussian ( http://www.socr.ucla.edu/Applets.dir/NormalApprox2PoissonApplet.html ): <br>
if X $\sim$ Poisson($\lambda$) $\Rightarrow$ X $\approx N(\mu = \lambda,\sigma = \sqrt\lambda )$ 


e.g. D_noise = SQRT ( D )

### Read out noise:
The read out noise is a Gaussian process, therefore can draw from a Gaussian distribution with a sigma that is the read out noise value given.  

> CCD manufacturers measure and report CCD noise as a number of electrons RMS (Root Mean Square).  You’ll typically see it presented like this, 15eˉ RMS, meaning that with this CCD, you should expect to see about 15 electrons of noise per pixel.  More precisely, 15eˉ RMS is the standard deviation around the mean pixel value. <br>
( http://www.qsimaging.com/ccd_noise.html )

It seems that we don't actually know the mean value of the read out noise, so we should center the read out noise at zero (since we can subtract the mean value off in the images).

Gaussian_noise = sigma = R

### Combining noise:
We then add the noise in quadrature (error propagation):

Noise_total = SQRT ( Poisson_noise ^ 2 + Gaussian_noise ^2 + ... ) = SQRT ( (meanvalue_Poisson) + R^2 + ...)

For a couple references on error propagation, check out:
http://www.mso.anu.edu.au/pfrancis/ObsTech/Stats2.pdf and https://www.deanza.edu/faculty/marshburnthomas/pdf/ErrorPropagation.pdf.


### Testing - G-Band Sky
https://arxiv.org/pdf/1211.4672.pdf
sky brightness in g-band is ~ 21.5 mag/arcsec^2.

The sky background in photons is calculated in SBUnitConversion.ipynb, and is predicted to be about 60 photons / m^2 / s / arcsec^2.

In [1]:
import numpy as np
import eagle_constants_and_units as c
import cosmo_utils as csu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits.axes_grid1 as axgrid
from astropy import constants as const
from astropy import units as u

import datetime

%matplotlib inline

In [2]:
%run 'load_data.ipynb'

Adding the sky background (and the Poisson noise background from the data as well, in one go).

In [5]:
def add_skybackground(data,B_sky,exptime,numpixel,debugging):
    'Sky Background'
    B_sky_inexptime = B_sky*exptime
    B_sky_total     = B_sky_inexptime*numpixel    
    B_sky_array = np.zeros((data.shape[0],data.shape[1]))
    for x in range(data.shape[0]):
        for y in range(data.shape[1]):
            B_sky_array[x][y]=np.random.normal(0,np.sqrt(B_sky_total+data[x][y])) 
#            B_sky_array[x][y]=np.random.poisson(B_sky_total)

    if debugging:
        print "DEBUGGING: the mean total background signal, B_sky_total [electrons], is: %s"\
                %B_sky_total
        print "DEBUGGING: the total background noisy signal [electrons] ranges from: %s to %s"\
                %(np.min(B_sky_array),np.max(B_sky_array))

    return B_sky_array,B_sky_total

Adding the dark current. 

In [12]:
def add_darkcurrent(data,D,exptime,numpixel,debugging,numlens,CASTOR=False):
    'Dark Current'
    D_total = D*exptime*numpixel*numlens
    
    if CASTOR:
        print "CASTOR: D_total = D*exptime*numpixel"
        D_total = D*exptime*numpixel
    
    if debugging:
        print "DEBUGGING: The dark current in one pixel of one camera is: %s electrons per second" % D
        print "DEBUGGING: The dark current in one pixel added over %s lenses and %s seconds is: %s"\
                %(numlens,exptime,D_total)
    
    D_array_total = np.zeros((data.shape[0],data.shape[1]))
    for x in range(data.shape[0]):
        for y in range(data.shape[1]):
            D_array_total[x][y]=np.random.normal(0,np.sqrt(D_total)) 
    
    if debugging:
        print "DEBUGGING: the total dark current noisy signal [electrons] ranges from: %s to %s"\
                %(np.min(D_array_total),np.max(D_array_total))
        
    return D_array_total, D_total

Adding the read out noise.

1. If the number of exposures is not provided, assume the exposures are an hour long, so get the number of times data read out by dividing exposure time by 3600 secounds
2. The read out noise is Gaussian distributed with sigma R (10e- for Dragonfly) so return a random value draw from a Gaussian distribution with sigma R

This algorithm for applying the readout noise assumes that each 2.8 arcsecond pixel (i.e. inherent Dragonfly pixel, not binned) is assigned a (random) readout noise.  

OLD: Since we will end up binning the data over some number of pixels, "numpixel", we build an array the size of the binned array, and in each index of the array, we store the SUM of the readout noise from each of the pixels and from each exposure.  We store the SUM instead of the AVERAGE because we won't be able to calculate the average readout noise independantly in the actual data (unless we do on-chip binning).  (So, right now I am assuming off-chip binning).

NEW: Since sum of gaussian drawn random variables is also a gaussian, replace this with drawing a single variable from a Gaussian with standard deviation that is quadrature-added standard deviation of first method.

$Z \sim N(0,numtotal \times R^2 )$

The binned read out noise is stored in "R_array".

In [10]:
def add_readoutnoise(data,R,numpixel,expnum,exptime,debugging,numlens,CASTOR=False):
    'ReadOut Noise'
    numexp = 1
    if expnum is None:
        numexp = exptime/3600. # hour long exposures
        print "VERBOSE: No expnum provided, assuming hour long exposures (numexp = %s)."%int(round(numexp))
    else:
        numexp = expnum
    
    numtotal = int(numpixel)*int(round(numexp))*int(round(numlens))
    
    if CASTOR:
        print "DEBUGGING: The number of exposures is: %s" % numexp
        print "CASTOR: numtotal = int(numpixel)*int(round(numexp))"
        numtotal = int(numpixel)*int(round(numexp))
    
    if debugging and not CASTOR:
        print "DEBUGGING: The number of exposures is: %s" % numexp
        print "DEBUGGING: Drawing %s values (numpixel=%s * numexp=%s * numlens=%s) per binned pixel and summing them." \
                %(numtotal,int(numpixel),round(numexp),round(numlens))
    
    R_squared_array = np.zeros((data.shape[0],data.shape[1]))
    R_array = np.zeros((data.shape[0],data.shape[1]))
    for x in range(data.shape[0]):
        for y in range(data.shape[1]):
            R_array[x][y]=np.random.normal(0,np.sqrt(numtotal*R**2))
            R_squared_array[x][y]=(R_array[x][y])**2 # centered at 0, st dev of R, return numpixel*numexp values
    
    if debugging:
        print "DEBUGGING: Standard deviation of R array: %s " % np.std(R_array)
        print "DEBUGGING: the max and min R values are: %s and %s" \
                %(np.max(R_array),np.min(R_array))
        
    return R_array, numtotal*R**2

In [46]:
def addnoise(data,resolution,includemeannoise=False,plotchecks = False,log = True,R=None,expnum=None, 
             filterwidth=3.0, exptime=10**3*3600.,CMOS=False, debugging=True, testing=False,CASTOR=False,
             numlens = 48.,D = 0.04):
    """
    This function adds noise to simulated data (such as the EAGLE simulation) to create mock observations.
    The EAGLE data is logged though, so the log = True for the EAGLE stuff
    """    
    # Dragonfly info
    #numlens = 48. 
    area_lens = np.pi*(14.3/2)**2 * numlens             # cm^2, 48 * 14.3 cm diameter lenses
    pix_size = 2.8                                      # arcsec
    pix_area_rad  = (pix_size * (1./206265.))**2        # rad^2, the pixel size of the CCD
    tau_l = 0.85                                        # transmittance of the Dragonfly lens
    tau_f = 1.                                          # transmittance of the Halpha filter -- assumed for now
    #D = 0.04       # dark current (electrons / s) 
    
    print "******* Adding noise to the input data to simulate a mock observation by the %s lens Dragonfly Telescope *******" %numlens
        
    binpix_size = resolution # arcsec
    numpixel = (round(binpix_size/pix_size))**2
    
    if debugging:
        print "DEBUGGING: the binpix_size (resolution) is %s" %binpix_size
        print "DEBUGGING: the pixel size (inherent) is %s" %pix_size
    
    if CMOS:
        print "VERBOSE: Using new CMOS cameras... (QE = 0.70, R = 2.)"
        QE = 0.70                                           # quantum efficiency of the CMOS detector
    else:
        print "VERBOSE: Using old cameras... (QE = 0.48, R = 10.)"
        QE = 0.48    
    
    if R is None:
        if CMOS:
            R = 2.                               # read noise (electrons)
        else:
            R = 10.                              
    
    
    if CASTOR:
        print ""
        print "Inputting CASTOR values"
        print ""
        area_lens = np.pi*(100./2.)**2 # cm^s
        pix_size  = 0.5       # arcsec
        pix_area_rad  = (pix_size * (1./206265.))**2  
        tau_l = 0.85 
        tau_f = 1.
        D = 0.04
        R = 2.
        QE = 0.7
        binpix_size = resolution # arcsec
        numpixel = (round(binpix_size/pix_size))**2

        print "area_lens=%s,pix_size=%s,tau_l=%s,tau_f=%s,D=%s,R=%s,QE=%s"%(area_lens,pix_size,tau_l,tau_f,D,R,QE)
        print ""
    
    if debugging:
        print "DEBUGGING: R is : %s" % R
        print "DEBUGGING: the number of pixels per bin is %s" % numpixel
    
    if log:
        if debugging:
            print "DEBUGGING: raise the data by 10** since was logged data before..."
        data = 10**data
    
    'total signal incident in exposure time'
    totsignal = data * exptime # ( photons / cm^2 /sr )
    'total signal detected (accounting for system efficiency)'
    detsignal = totsignal * QE * tau_l * tau_f * area_lens * pix_area_rad * numpixel
    
    if plotchecks:
        fig, (ax1,ax2,ax3) = plt.subplots(1, 3, figsize=(16, 4))
        ax1.hist(np.ravel(np.log10(data)),edgecolor='black', alpha = 0.5,bins=50)
        ax1.set_title('log (raw data)')
        ax1.set_xlabel('log (incident photons / cm^2 /sr / s)')
        ax2.hist(np.ravel(np.log10(totsignal)),edgecolor='black', alpha = 0.5,bins=50)
        ax2.set_title('log (total signal)')
        ax2.set_xlabel('log (incident photons per (binned) pixel)')
        ax3.hist(np.ravel(np.log10(detsignal)),edgecolor='black', alpha = 0.5,bins=50)
        ax3.set_title('log (detected signal)')
        ax3.set_xlabel('log (detected photons per (binned) pixel)')

    if debugging:
        print "DEBUGGING: the total object signal [electrons] detected ranges from: %s to %s"%(np.min(detsignal),np.max(detsignal))
        #print "DEBUGGING: an example of the object signal [electrons] is: %s"%detsignal[0]

    print "The width of the filter that the data was taken with is %s nm. " % (filterwidth)

    if not CASTOR:
        'load background value: '
        filtdict = {1.0:0.560633,3.0:1.473626,'SloanG':60.0}
        if filterwidth in filtdict:
            B = filtdict[filterwidth]
            if debugging:
                print "DEBUGGING: Sky background in filter width of %s nm is calculated already: %s."%(filterwidth,B)
        else:
            B = getBackground(656.3,656.3+filterwidth,machine,plot=True)
            print "Sky background in filter width of %s nm is calculated to be: %s."%(filterwidth,B)
    elif CASTOR:
        B = 0.
        print "Sky background is zero."
    
    print "Adding sky background noise and shot noise..."
    'background sky signal detected [B]=ph/s/arcsec^2/m^2, [B_sky]=e-/s (in a pixel)'
    B_sky = B * QE * tau_l * tau_f * area_lens*(1/100.)**2 * pix_size**2
    if debugging:
        print "DEBUGGING: the background in the bandwidth is: %s photon/s/arcsec^2/m^2"%B
        print "DEBUGGING: the background signal, B_sky, is: %s electron/s/pixel"%B_sky

    B_sky_array,B_sky_total = add_skybackground(detsignal,B_sky,exptime,numpixel,debugging)
    noiseadded_signal = detsignal + B_sky_array #+ B_sky*exptime*numpixel  ## last term is constant (mean sky background - don't really need to add in...)
    
    if CASTOR:
        noiseadded_signal = detsignal
    
    if debugging:
        print "DEBUGGING: added the mean background sky noise: %s "%(B_sky*exptime*numpixel)
    
    if R != 0:
        print "Adding read out noise to the signal..."
        R_array,R_total     = add_readoutnoise(detsignal,R,numpixel,expnum,exptime,debugging,numlens,CASTOR=CASTOR)
    else:
        print "Setting read out noise to zero in whole array... "
        R_array = np.zeros((detsignal.shape[0],detsignal.shape[1]))
        R_total = 0.

    noise_from_detector = R_array #+ R*numpixel*expnum*numlens
        
    print "Adding dark current to the signal..."
    D_array,D_total     = add_darkcurrent(detsignal,D,exptime,numpixel,debugging,numlens,CASTOR=CASTOR)
    noise_from_detector = noise_from_detector + D_array
        
    noiseadded_signal = noiseadded_signal + noise_from_detector
    
    if includemeannoise:
        print "Adding on the mean value of the noise (R = %s, D = %s, B = %s)" % (R_total,D_total,B_sky_total)
        noiseadded_signal = noiseadded_signal + B_sky_total + R_total + D_total
    
    if plotchecks:
        fig, (ax1,ax2,ax3) = plt.subplots(1, 3, figsize=(16, 4))
        ax1.hist(np.ravel(np.log10(B_sky_array[B_sky_array>0])),edgecolor='black', alpha = 0.5,bins=50)
        ax1.set_title('log (sky background noise)')
        ax1.set_xlabel('log ( photons / binned pixel)')
        ax2.hist(np.ravel(np.log10(R_array[R_array>0])),edgecolor='black', alpha = 0.5,bins=50)
        ax2.set_title('log (ro noise)')
        ax2.set_xlabel('log ( photons / binned pixel)')
        ax3.hist(np.ravel(np.log10(D_array[D_array>0])),edgecolor='black', alpha = 0.5,bins=50)
        ax3.set_title('log (dark current noise)')
        ax3.set_xlabel('log ( photons / binned pixel)')
  
        fig, (ax1,ax2,ax3) = plt.subplots(1, 3, figsize=(16, 4))
        ax1.hist(np.ravel((B_sky_array[B_sky_array>0])),edgecolor='black', alpha = 0.5,bins=50)
        ax1.set_title(' (sky background noise)')
        ax1.set_xlabel(' ( photons / binned pixel)')
        ax2.hist(np.ravel((R_array[R_array>0])),edgecolor='black', alpha = 0.5,bins=50)
        ax2.set_title(' (ro noise)')
        ax2.set_xlabel(' ( photons / binned pixel)')
        ax3.hist(np.ravel((D_array[D_array>0])),edgecolor='black', alpha = 0.5,bins=50)
        ax3.set_title(' (dark current noise)')
        ax3.set_xlabel(' ( photons / binned pixel)')
        
    if plotchecks:
        fig, (ax1) = plt.subplots(1, 1, figsize=(6, 6))
        ax1.hist(np.ravel(np.log10(B_sky_array[B_sky_array>0])),edgecolor='black', alpha = 1.,bins=50,label='sky noise')
        ax1.hist(np.ravel(np.log10(R_array[R_array>0])),edgecolor='black', alpha = 0.8,bins=50,label='readout noise')
        ax1.hist(np.ravel(np.log10(D_array[D_array>0])),edgecolor='black', alpha = 0.5,bins=50,label='dark current noise')
        ax1.hist(np.ravel(np.log10(detsignal)),edgecolor='black', alpha = 0.3,bins=50,label = 'detected signal')
        plt.legend(title='in log space')
    
    print "******* Finished. *******"
    print ""
    
    return noiseadded_signal,B_sky_array,R_array,D_array

In [75]:
def getBackground(start,end,machine,plot=False):
    # Returns the total background flux in the wavlength interval supplied i.e. returns (flux)*(wavlength interval) 
    wavelength = []
    flux = []
    
    if machine=='chinook':
        geminiloc='/Users/lokhorst/Documents/Eagle/Gemini_skybackground.dat'
    elif machine=='coho':
        geminiloc='/Users/deblokhorst/Documents/Dragonfly/HalphaScripts/Gemini_skybackground.dat'
    
    with open(geminiloc,'r') as f:  #wavelength in nm, flux in phot/s/nm/arcsec^2/m^2
        for line in f:
            if line[0]!='#' and len(line)>5:
                tmp = line.split()
                wavelength.append(tmp[0])
                flux.append(tmp[1])
                
    wavelength = np.array(wavelength,'d')
    flux = np.array(flux,'d')
    
    start_ind = (np.abs(wavelength-start)).argmin()
    end_ind   = (np.abs(wavelength-end)).argmin()
    
    # if spacings are not even, need to add element by element
    total=0
    for index in np.arange(start_ind,end_ind):
      #  print index,index+1
      #  print total
        total = total+(flux[index]*(wavelength[index+1]-wavelength[index]))
        
    # if spacings are even, can just take the average of the flux array and times it by the total bandwidth
    np.mean(flux[start_ind:end_ind])*(wavelength[end_ind]-wavelength[start_ind])
    
   # print('start index and end index: %s and %s'%(start_ind,end_ind))
   # print(wavelength[start_ind:end_ind]-wavelength[start_ind+1:end_ind+1])
    if plot==True:
        plt.plot(wavelength[start_ind-100:end_ind+100],flux[start_ind-100:end_ind+100])
        top = max(flux[start_ind-100:end_ind+100])
        plt.plot([start,start,end,end,start],[0,top,top,0,0])
        plt.show()
        
    return total

In [ ]:
### EXAMPLE ###
## exposure time
#exptime = 60.*60.*10**5  # seconds  (10^5 hours)
## where is the data
#machine='coho'
## what distance and resolution do you want
#distance = '50Mpc'; resolution = 100
#data_tuple = loaddata(machine=machine,resolution=resolution,distance=distance)
## or do you want the raw data
#data_tuple = loaddata(machine=machine)
#data = data_tuple[0]